In [ ]:
import numpy as np
# from biom.table import Table
# from biom import load_table
import pandas as pd
import os
from utils.ncbi.names import generate_names_df, names_db_path, standardize_core
from utils.ncbi.jams_convert import fix_name, convert_jams_to_taxid
from utils.data_paths import *

In [ ]:
# Use the TEDreadsTAX reports in TAXprofiles/TEDreadsTAX/reports.
data = hmpTongue.wgsa
output_path = "pipelines/hmp/tongue/wgsa2"

In [ ]:
# The genus was changed, but the Amos paper uses the original genus name.
# replacement_dict = {"Anaerobutyricum hallii": "Eubacterium hallii", "Anaerobutyricum": "Eubacterium"}
# For the Amos paper, the genus was changed to Eubacterium. This is the only result, so we can just replace it.

In [ ]:
def standardize_wgsa(df: pd.DataFrame):
    """
    Standardizes the WGSA data using the standardize_core function.
    """
    names_df = generate_names_df(names_db_path, load_pickle=True)

    # We can use the convert_jams_to_taxid function from the utils.ncbi.convert_jams since the format is the same.
    ann, unann = convert_jams_to_taxid(df.head(50), names_df)

    return ann, unann

In [ ]:
tax_dict = {"Genus": "G", "Species": "S", "Family": "F", "Order": "O", "Class": "C", "Phylum": "P", "Kingdom": "K"}

def clean_and_parse_wgsa(data_path, output_dir, rank="Genus", left_prefix=""):
    df = pd.read_csv(data_path, sep="\t", header=None, usecols=[1, 3, 4, 5])

    # Split off the first two rows.
    total_counts = df.iloc[:2, 0].sum()

    df = df.where(df[3] == tax_dict[rank]).dropna()
    df.sort_values(by=1, ascending=False, inplace=True)

    clean_genus = df[[5, 1, 4]].copy(deep=True)
    clean_genus.columns = [rank, "RA", "TAX_ID"]

    # Convert the RA column from counts to RA.
    clean_genus["RA"] = clean_genus["RA"].apply(lambda x: x / total_counts)

    clean_genus.set_index(rank, inplace=True)

    indices = clean_genus.index
    indices = [i.lstrip() for i in indices]

    # Remove any [ and ] characters from the indices.
    indices = [i.replace("[", "") for i in indices]
    indices = [i.replace("]", "") for i in indices]

    clean_genus.index = indices

    # Since the WGSA data already contains the TAXID, it is much faster than this function.
    # ann, uann = standardize_wgsa(clean_genus)

    clean_genus = clean_genus.astype({"TAX_ID": int})

    prefix = os.path.basename(data_path).split("_")[0]
    # left_prefix = "s"
    output_file = os.path.join(output_dir, left_prefix + prefix + "_" + rank.lower() + "_" + "relabund_annotated.csv")

    clean_genus.to_csv(output_file, sep=",", header=True, index_label=rank)

# clean_and_parse_wgsa(cami_sim_data, output_path)

In [ ]:
# There may be more than one output file, so we need to combine them.
def combine_files(data_path: str, rank: str):
    if not os.path.exists(data_path):
        raise FileNotFoundError("The data path does not exist.")

    for root, dirs, files in os.walk(data_path):
        print(files)
        if len(files) == 0:
            raise Exception("No files found in output directory.")
            
        for file in files:
            if "REPORT" in file:
                print(os.path.join(root, file))
                clean_and_parse_wgsa(os.path.join(root, file), output_path, rank=rank)

combine_files(data, "Genus")
combine_files(data, "Species")